In [2]:
!pip install faiss-cpu sentence-transformers datasets transformers


  Using cached faiss_cpu-1.12.0.tar.gz (69 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss._swigfaiss' extension
      swigging third-party/faiss/faiss/python/swigfaiss.i to third-party/faiss/faiss/python/swigfaiss_wrap.cpp
   

In [ ]:
import pickle
from datasets import Dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Загружаем твой датасет
with open("data.pickle", "rb") as f:
    data = pickle.load(f)

texts = [
    f"Question: {ex['question']}\nOptions: {ex['options']}\nAnswer: {ex['answer']}"
    for ex in data
]

encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embs = encoder.encode(texts, convert_to_numpy=True)

d = embs.shape[1]     
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(embs)

print("FAISS index built:", faiss_index.ntotal)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Загружаем Qwen
model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto").eval()

def rag_answer(question):
    query = f"Question: {question}"
    
    # 1. Получить эмбеддинг вопроса
    q_emb = encoder.encode([query], convert_to_numpy=True)

    # 2. Найти похожие куски (top-3)
    D, I = faiss_index.search(q_emb, 3)

    retrieved = "\n\n".join([texts[i] for i in I[0]])

    # 3. Сформировать промпт с контекстом
    prompt = f"""
You are an expert at solving math and physics multiple-choice questions.

Use ONLY the context below to answer.

### Context:
{retrieved}

### Question:
{question}

### Your answer (just letters without spaces):
"""
    
    # 4. Генерация
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=100)

    answer = tokenizer.decode(out[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
    
    return answer.strip()


In [ ]:
print(rag_answer("The perimeter of a square with side 3 is:"))


In [ ]:
from rank_bm25 import BM25Okapi

# подготавливаем токены
tokenized_docs = [doc.split() for doc in docs]
bm25 = BM25Okapi(tokenized_docs)

def retrieve_bm25(query, top_k=3):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    ranked = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    return [docs[i] for i in ranked[:top_k]]

# пример
question = "What is the formula for the perimeter of a square?"
retrieved = retrieve_bm25(question)
answer = generate_answer(question, retrieved)

print("=== BM25 Retrieved ===")
print(retrieved)
print("=== Answer ===")
print(answer)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import json

# 1. Загрузка моделей
print("Загружаем модели...")

# Модель для эмбеддингов (русский язык)
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Модель для генерации
model_name = "IlyaGusev/saiga_yandexgpt_8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Подготовка базы знаний (пример с русскими текстами)
def create_knowledge_base():
    """Создаем расширенную базу знаний на русском"""
    documents = [
        # География и города
        "Москва - столица России, крупнейший город страны с населением более 12 миллионов человек. Основана в 1147 году Юрием Долгоруким.",
        "Санкт-Петербург был основан Петром I в 1703 году и является культурной столицей России. Население около 5 миллионов человек.",
        "Новосибирск - третий по численности населения город России, крупнейший научный и образовательный центр Сибири.",
        "Сочи - курортный город на черноморском побережье России, известный своими пляжами и горнолыжными курортами.",
        "Калининград - самый западный город России, расположенный между Польшей и Литвой на берегу Балтийского моря.",
        
        # Природа и география
        "Байкал - самое глубокое озеро в мире, расположенное в Сибири. Глубина достигает 1642 метров, содержит 20% мировых запасов пресной воды.",
        "Волга - самая длинная река в Европе, протяженностью 3530 км. Протекает через 15 субъектов Российской Федерации.",
        "Эльбрус - высочайшая горная вершина России и Европы, высота 5642 метра. Расположен на Кавказе.",
        "Камчатка - полуостров на Дальнем Востоке России, известный своими вулканами и гейзерами.",
        "Тайга - крупнейший в мире лесной массив, занимающий большую часть Сибири и Дальнего Востока.",
        
        # История
        "Великая Отечественная война длилась с 1941 по 1945 год и завершилась победой Советского Союза над нацистской Германией.",
        "Октябрьская революция 1917 года привела к установлению советской власти в России и созданию СССР.",
        "Петр I Великий - русский царь, провел масштабные реформы и основал Санкт-Петербург. Правил с 1682 по 1725 год.",
        "Екатерина II Великая - императрица России с 1762 по 1796 год, значительно расширила территорию Российской империи.",
        
        # Культура и искусство
        "Эрмитаж в Санкт-Петербурге - один из крупнейших художественных музеев мира, основанный Екатериной II в 1764 году.",
        "Третьяковская галерея в Москве - главный музей русского изобразительного искусства, основанный Павлом Третьяковым.",
        "Большой театр в Москве - один из ведущих театров оперы и балета в мире, основанный в 1776 году.",
        "Александр Пушкин - великий русский поэт, основоположник современного русского литературного языка. Родился в 1799 году.",
        "Лев Толстой - классик русской литературы, автор романов 'Война и мир' и 'Анна Каренина'.",
        
        # Наука и технологии
        "Дмитрий Менделеев - русский ученый, создатель периодической системы химических элементов в 1869 году.",
        "Юрий Гагарин - первый человек в космосе, совершил полет 12 апреля 1961 года на корабле 'Восток-1'.",
        "Михаил Ломоносов - русский ученый-энциклопедист, основатель Московского университета в 1755 году.",
        "Сергей Королев - советский конструктор ракетно-космической техники, руководитель программы первого полета человека в космос.",
        
        # Экономика и промышленность
        "Газпром - крупнейшая газовая компания мира, основана в 1989 году. Штаб-квартира расположена в Москве.",
        "Роснефть - одна из крупнейших нефтяных компаний мира, основана в 1993 году.",
        "Транссибирская магистраль - самая длинная железная дорога в мире, протяженностью 9288 км от Москвы до Владивостока.",
        "Рубль - национальная валюта России, введена в обращение в 14 веке. Современный код валюты - RUB.",
        
        # Политика
        "Владимир Путин является президентом Российской Федерации с 2012 года. Родился в 1952 году в Ленинграде.",
        "Государственная Дума - нижняя палата парламента России, состоит из 450 депутатов.",
        "Конституция России была принята 12 декабря 1993 года и является основным законом страны.",
        "Кремль - исторический комплекс в Москве, резиденция президента России. Построен в 15 веке.",
        
        # Образование
        "МГУ имени Ломоносова - старейший и крупнейший университет России, основан в 1755 году.",
        "ЕГЭ - единый государственный экзамен, обязательный для всех выпускников школ России с 2009 года.",
        "Российская академия наук - главная научная организация страны, основана Петром I в 1724 году.",
        
        # Спорт
        "Хоккей с шайбой - один из самых популярных видов спорта в России. Сборная России многократный чемпион мира.",
        "Фигурное катание - традиционно сильный вид спорта для России, множество олимпийских чемпионов.",
        "Футбольный клуб Зенит из Санкт-Петербурга - один из ведущих футбольных клубов России.",
        "Олимпиада 1980 года проводилась в Москве, была первой Олимпиадой в социалистической стране.",
        
        # Традиции и праздники
        "День Победы отмечается 9 мая в честь победы в Великой Отечественной войне. Главный парад проходит на Красной площади.",
        "Масленица - традиционный русский праздник проводов зимы, отмечается за неделю до Великого поста.",
        "Новый год - главный праздник в России, отмечается в ночь с 31 декабря на 1 января.",
        "День России - государственный праздник 12 июня, отмечается с 1992 года."
    ]
    return documents

# Примеры вопросов для тестирования
test_questions = [
    "Какая самая длинная река в Европе?",
    "Когда был основан Санкт-Петербург?",
    "Кто был первым человеком в космосе?",
    "Сколько человек живет в Москве?",
    "Какое озеро самое глубокое в мире?",
    "Кто создал периодическую таблицу элементов?",
    "Как называется валюта России?",
    "Когда была Октябрьская революция?",
    "Какой университет самый старый в России?",
    "Что такое Эрмитаж?",
    "Кто является президентом России?",
    "Какой праздник отмечается 9 мая?",
    "Сколько километров составляет Транссибирская магистраль?",
    "Кто автор романа 'Война и мир'?",
    "В каком году Юрий Гагарин полетел в космос?"
]

def run_test_questions():
    """Функция для тестирования системы на подготовленных вопросах"""
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("=" * 50)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ")
    print("=" * 50)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Вопрос: {question}")
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"Ответ: {answer}")
            print(f"Использовано документов: {len(context_docs)}")
            print("-" * 30)
        except Exception as e:
            print(f"Ошибка: {e}")
            print("-" * 30)


# 3. Создание векторной базы данных
def create_vector_db(documents):
    """Создаем векторную базу данных"""
    print("Создаем векторную базу...")
    
    # Получаем эмбеддинги для документов
    embeddings = embedding_model.encode(documents)
    
    # Создаем FAISS индекс
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (косинусное сходство)
    
    # Нормализуем векторы для косинусного сходства
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    return index, documents

# 4. Поиск релевантных документов
def search_similar_documents(query, index, documents, top_k=3):
    """Ищем наиболее релевантные документы для запроса"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Ищем похожие документы
    scores, indices = index.search(query_embedding, top_k)
    
    # Возвращаем найденные документы и их скоринги
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):
            results.append({
                'text': documents[idx],
                'score': scores[0][i]
            })
    
    return results

# 5. Создание промпта с контекстом
def create_prompt(question, context_docs):
    """Создаем промпт для модели с контекстом"""
    context = "\n".join([f"- {doc['text']}" for doc in context_docs])
    
    prompt = f"""Контекст:
{context}

Вопрос: {question}

Ответ: """
    
    return prompt

# 6. Генерация ответа
def generate_answer(prompt):
    """Генерируем ответ с помощью модели"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Убираем промпт из ответа
    answer = response[len(prompt):].strip()
    
    return answer

# 7. Основная функция RAG
def rag_pipeline(question, index, documents):
    """Полный пайплайн RAG"""
    print(f"Вопрос: {question}")
    
    # Шаг 1: Поиск релевантных документов
    print("Ищем релевантные документы...")
    relevant_docs = search_similar_documents(question, index, documents)
    
    print("Найденные документы:")
    for i, doc in enumerate(relevant_docs):
        print(f"{i+1}. {doc['text']} (сходство: {doc['score']:.3f})")
    
    # Шаг 2: Создание промпта
    prompt = create_prompt(question, relevant_docs)
    
    # Шаг 3: Генерация ответа
    print("\nГенерируем ответ...")
    answer = generate_answer(prompt)
    
    return answer, relevant_docs

# 8. Запуск системы
def main():
    # Инициализация
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("RAG система готова! Введите ваш вопрос (или 'выход' для завершения):")
    
    # Интерактивный цикл
    while True:
        question = input("\nВаш вопрос: ").strip()
        
        if question.lower() in ['выход', 'exit', 'quit']:
            break
            
        if not question:
            continue
            
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"\nОтвет: {answer}")
            print(f"\nИсточники: {len(context_docs)} документов")
            
        except Exception as e:
            print(f"Ошибка: {e}")

if __name__ == "__main__":
    run_test_questions()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import json

# 1. Загрузка моделей
print("Загружаем модели...")

# Модель для эмбеддингов (русский язык)
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Модель для генерации
model_name = "IlyaGusev/saiga_yandexgpt_8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Подготовка базы знаний (пример с русскими текстами)
def create_knowledge_base():
    """Создаем расширенную базу знаний на русском"""
    documents = [
        # География и города
        "Москва - столица России, крупнейший город страны с населением более 12 миллионов человек. Основана в 1147 году Юрием Долгоруким.",
        "Санкт-Петербург был основан Петром I в 1703 году и является культурной столицей России. Население около 5 миллионов человек.",
        "Новосибирск - третий по численности населения город России, крупнейший научный и образовательный центр Сибири.",
        "Сочи - курортный город на черноморском побережье России, известный своими пляжами и горнолыжными курортами.",
        "Калининград - самый западный город России, расположенный между Польшей и Литвой на берегу Балтийского моря.",
        
        # Природа и география
        "Байкал - самое глубокое озеро в мире, расположенное в Сибири. Глубина достигает 1642 метров, содержит 20% мировых запасов пресной воды.",
        "Волга - самая длинная река в Европе, протяженностью 3530 км. Протекает через 15 субъектов Российской Федерации.",
        "Эльбрус - высочайшая горная вершина России и Европы, высота 5642 метра. Расположен на Кавказе.",
        "Камчатка - полуостров на Дальнем Востоке России, известный своими вулканами и гейзерами.",
        "Тайга - крупнейший в мире лесной массив, занимающий большую часть Сибири и Дальнего Востока.",
        
        # История
        "Великая Отечественная война длилась с 1941 по 1945 год и завершилась победой Советского Союза над нацистской Германией.",
        "Октябрьская революция 1917 года привела к установлению советской власти в России и созданию СССР.",
        "Петр I Великий - русский царь, провел масштабные реформы и основал Санкт-Петербург. Правил с 1682 по 1725 год.",
        "Екатерина II Великая - императрица России с 1762 по 1796 год, значительно расширила территорию Российской империи.",
        
        # Культура и искусство
        "Эрмитаж в Санкт-Петербурге - один из крупнейших художественных музеев мира, основанный Екатериной II в 1764 году.",
        "Третьяковская галерея в Москве - главный музей русского изобразительного искусства, основанный Павлом Третьяковым.",
        "Большой театр в Москве - один из ведущих театров оперы и балета в мире, основанный в 1776 году.",
        "Александр Пушкин - великий русский поэт, основоположник современного русского литературного языка. Родился в 1799 году.",
        "Лев Толстой - классик русской литературы, автор романов 'Война и мир' и 'Анна Каренина'.",
        
        # Наука и технологии
        "Дмитрий Менделеев - русский ученый, создатель периодической системы химических элементов в 1869 году.",
        "Юрий Гагарин - первый человек в космосе, совершил полет 12 апреля 1961 года на корабле 'Восток-1'.",
        "Михаил Ломоносов - русский ученый-энциклопедист, основатель Московского университета в 1755 году.",
        "Сергей Королев - советский конструктор ракетно-космической техники, руководитель программы первого полета человека в космос.",
        
        # Экономика и промышленность
        "Газпром - крупнейшая газовая компания мира, основана в 1989 году. Штаб-квартира расположена в Москве.",
        "Роснефть - одна из крупнейших нефтяных компаний мира, основана в 1993 году.",
        "Транссибирская магистраль - самая длинная железная дорога в мире, протяженностью 9288 км от Москвы до Владивостока.",
        "Рубль - национальная валюта России, введена в обращение в 14 веке. Современный код валюты - RUB.",
        
        # Политика
        "Владимир Путин является президентом Российской Федерации с 2012 года. Родился в 1952 году в Ленинграде.",
        "Государственная Дума - нижняя палата парламента России, состоит из 450 депутатов.",
        "Конституция России была принята 12 декабря 1993 года и является основным законом страны.",
        "Кремль - исторический комплекс в Москве, резиденция президента России. Построен в 15 веке.",
        
        # Образование
        "МГУ имени Ломоносова - старейший и крупнейший университет России, основан в 1755 году.",
        "ЕГЭ - единый государственный экзамен, обязательный для всех выпускников школ России с 2009 года.",
        "Российская академия наук - главная научная организация страны, основана Петром I в 1724 году.",
        
        # Спорт
        "Хоккей с шайбой - один из самых популярных видов спорта в России. Сборная России многократный чемпион мира.",
        "Фигурное катание - традиционно сильный вид спорта для России, множество олимпийских чемпионов.",
        "Футбольный клуб Зенит из Санкт-Петербурга - один из ведущих футбольных клубов России.",
        "Олимпиада 1980 года проводилась в Москве, была первой Олимпиадой в социалистической стране.",
        
        # Традиции и праздники
        "День Победы отмечается 9 мая в честь победы в Великой Отечественной войне. Главный парад проходит на Красной площади.",
        "Масленица - традиционный русский праздник проводов зимы, отмечается за неделю до Великого поста.",
        "Новый год - главный праздник в России, отмечается в ночь с 31 декабря на 1 января.",
        "День России - государственный праздник 12 июня, отмечается с 1992 года."
    ]
    return documents

# Примеры вопросов для тестирования
test_questions = [
    "Какая самая длинная река в Европе?",
    "Когда был основан Санкт-Петербург?",
    "Кто был первым человеком в космосе?",
    "Сколько человек живет в Москве?",
    "Какое озеро самое глубокое в мире?",
    "Кто создал периодическую таблицу элементов?",
    "Как называется валюта России?",
    "Когда была Октябрьская революция?",
    "Какой университет самый старый в России?",
    "Что такое Эрмитаж?",
    "Кто является президентом России?",
    "Какой праздник отмечается 9 мая?",
    "Сколько километров составляет Транссибирская магистраль?",
    "Кто автор романа 'Война и мир'?",
    "В каком году Юрий Гагарин полетел в космос?"
]

def run_test_questions():
    """Функция для тестирования системы на подготовленных вопросах"""
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("=" * 50)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ")
    print("=" * 50)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Вопрос: {question}")
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"Ответ: {answer}")
            print(f"Использовано документов: {len(context_docs)}")
            print("-" * 30)
        except Exception as e:
            print(f"Ошибка: {e}")
            print("-" * 30)


# 3. Создание векторной базы данных
def create_vector_db(documents):
    """Создаем векторную базу данных"""
    print("Создаем векторную базу...")
    
    # Получаем эмбеддинги для документов
    embeddings = embedding_model.encode(documents)
    
    # Создаем FAISS индекс
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (косинусное сходство)
    
    # Нормализуем векторы для косинусного сходства
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    return index, documents

# 4. Поиск релевантных документов
def search_similar_documents(query, index, documents, top_k=3):
    """Ищем наиболее релевантные документы для запроса"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Ищем похожие документы
    scores, indices = index.search(query_embedding, top_k)
    
    # Возвращаем найденные документы и их скоринги
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):
            results.append({
                'text': documents[idx],
                'score': scores[0][i]
            })
    
    return results

# 5. Создание промпта с контекстом
def create_prompt(question, context_docs):
    """Создаем промпт для модели с контекстом"""
    context = "\n".join([f"- {doc['text']}" for doc in context_docs])
    
    prompt = f"""Контекст:
{context}

Вопрос: {question}

Ответ: """
    
    return prompt

# 6. Генерация ответа
def generate_answer(prompt):
    """Генерируем ответ с помощью модели"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Убираем промпт из ответа
    answer = response[len(prompt):].strip()
    
    return answer

# 7. Основная функция RAG
def rag_pipeline(question, index, documents):
    """Полный пайплайн RAG"""
    print(f"Вопрос: {question}")
    
    # Шаг 1: Поиск релевантных документов
    print("Ищем релевантные документы...")
    relevant_docs = search_similar_documents(question, index, documents)
    
    print("Найденные документы:")
    for i, doc in enumerate(relevant_docs):
        print(f"{i+1}. {doc['text']} (сходство: {doc['score']:.3f})")
    
    # Шаг 2: Создание промпта
    prompt = create_prompt(question, relevant_docs)
    
    # Шаг 3: Генерация ответа
    print("\nГенерируем ответ...")
    answer = generate_answer(prompt)
    
    return answer, relevant_docs

# 8. Запуск системы
def main():
    # Инициализация
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("RAG система готова! Введите ваш вопрос (или 'выход' для завершения):")
    
    # Интерактивный цикл
    while True:
        question = input("\nВаш вопрос: ").strip()
        
        if question.lower() in ['выход', 'exit', 'quit']:
            break
            
        if not question:
            continue
            
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"\nОтвет: {answer}")
            print(f"\nИсточники: {len(context_docs)} документов")
            
        except Exception as e:
            print(f"Ошибка: {e}")

if __name__ == "__main__":
    run_test_questions()

In [ ]:
# quick_test.py - быстрый тест системы
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datasets import load_dataset

def quick_rag_test():
    """Быстрый тест RAG системы с реальными новостями"""
    
    print("🚀 Быстрая загрузка RAG системы...")
    
    # Загрузка моделей
    embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    
    tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_yandexgpt_8b")
    model = AutoModelForCausalLM.from_pretrained(
        "IlyaGusev/saiga_yandexgpt_8b",
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # Загрузка датасета
    print("📰 Загружаем новости...")
    dataset = load_dataset("IlyaGusev/ru_news", split="train[:500]")
    documents = []
    
    for item in dataset:
        text = item.get('text', '')
        title = item.get('title', '')
        if title and text:
            documents.append(f"{title}. {text[:200]}...")
    
    print(f"✅ Загружено {len(documents)} новостей")
    
    # Создание векторной базы
    embeddings = embedding_model.encode(documents)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    # Простые тестовые вопросы
    questions = [
        "Какие экономические новости?",
        "Что нового в политике?",
        "Какие культурные события?",
        "Какие спортивные новости?"
    ]
    
    print("\n🧪 Тестируем систему...")
    
    for question in questions:
        print(f"\n❓ Вопрос: {question}")
        
        # Поиск релевантных документов
        query_embedding = embedding_model.encode([question])
        faiss.normalize_L2(query_embedding)
        scores, indices = index.search(query_embedding, 2)
        
        # Создание контекста
        context = "\n".join([f"- {documents[idx]}" for idx in indices[0] if idx < len(documents)])
        
        # Генерация ответа
        prompt = f"""<bos><start_of_turn>user
На основе новостей ответь на вопрос:

{context}

Вопрос: {question}

Краткий ответ:<end_of_turn>
<start_of_turn>model
"""
        
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.7
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=False)
        if "<start_of_turn>model" in response:
            answer = response.split("<start_of_turn>model")[-1].replace("<end_of_turn>", "").strip()
            print(f"🤖 Ответ: {answer}")

if __name__ == "__main__":
    quick_rag_test()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import json
from datasets import load_dataset

# 1. Загрузка моделей
print("Загружаем модели...")

# Модель для эмбеддингов (русский язык)
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Модель для генерации
model_name = "IlyaGusev/saiga_yandexgpt_8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Загрузка реального датасета новостей
def create_knowledge_base():
    """Загружаем реальный датасет новостей на русском языке"""
    print("Загружаем датасет IlyaGusev/ru_news...")
    
    try:
        # Загружаем датасет
        dataset = load_dataset("IlyaGusev/ru_news", split="train[:1000]")  # Берем первые 1000 новостей
        
        documents = []
        for item in dataset:
            # Извлекаем текст новости
            text = item.get('text', '')
            title = item.get('title', '')
            
            # Создаем документ из заголовка и начала текста
            if title and text:
                document = f"{title}. {text[:300]}..."  # Ограничиваем длину
                documents.append(document)
            elif text:
                documents.append(text[:400] + "...")
            elif title:
                documents.append(title)
        
        print(f"Загружено {len(documents)} новостей из датасета")
        return documents
        
    except Exception as e:
        print(f"Ошибка загрузки датасета: {e}")
        # Резервный вариант - возвращаем несколько примеров
        return [
            "Россия и Китай усиливают экономическое сотрудничество в условиях санкций.",
            "Центробанк России сохранил ключевую ставку на прежнем уровне.",
            "В Москве открылась новая ветка метро, соединяющая центр с спальными районами.",
            "Ученые разработали новую вакцину от гриппа с повышенной эффективностью.",
            "Цены на нефть выросли на мировых рынках из-за сокращения добычи.",
        ]

# Примеры вопросов для тестирования (адаптированные под новостную тематику)
test_questions = [
    "Какие последние новости о экономике России?",
    "Что происходит с курсом рубля?",
    "Какие новые проекты в метро Москвы?",
    "Какие научные разработки последнего времени?",
    "Как изменились цены на нефть?",
    "Какие международные отношения у России?",
    "Что нового в банковской системе?",
    "Какие события в культурной жизни?",
    "Какие спортивные новости?",
    "Что происходит в сфере технологий?",
    "Какие изменения в образовании?",
    "Какие медицинские новости?",
    "Какие политические события?",
    "Что нового в строительстве?",
    "Какие экологические проблемы обсуждаются?"
]

def run_test_questions():
    """Функция для тестирования системы на подготовленных вопросах"""
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("=" * 60)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ С РЕАЛЬНЫМИ НОВОСТЯМИ")
    print("=" * 60)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Вопрос: {question}")
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"🤖 Ответ: {answer}")
            print(f"📊 Использовано документов: {len(context_docs)}")
            print("📰 Источники:")
            for j, doc in enumerate(context_docs, 1):
                print(f"   {j}. {doc['text'][:100]}... (сходство: {doc['score']:.3f})")
            print("-" * 80)
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            print("-" * 80)

# 3. Создание векторной базы данных
def create_vector_db(documents):
    """Создаем векторную базу данных"""
    print("Создаем векторную базу из новостей...")
    
    # Получаем эмбеддинги для документов
    embeddings = embedding_model.encode(documents)
    
    # Создаем FAISS индекс
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (косинусное сходство)
    
    # Нормализуем векторы для косинусного сходства
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    print(f"Векторная база создана: {len(documents)} документов")
    return index, documents

# 4. Поиск релевантных документов
def search_similar_documents(query, index, documents, top_k=3):
    """Ищем наиболее релевантные документы для запроса"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Ищем похожие документы
    scores, indices = index.search(query_embedding, top_k)
    
    # Возвращаем найденные документы и их скоринги
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):
            results.append({
                'text': documents[idx],
                'score': scores[0][i]
            })
    
    return results

# 5. Создание промпта с контекстом
def create_prompt(question, context_docs):
    """Создаем промпт для модели с контекстом"""
    context = "\n".join([f"- {doc['text']}" for doc in context_docs])
    
    prompt = f"""<bos><start_of_turn>user
На основе следующих новостных материалов ответь на вопрос:

{context}

Вопрос: {question}

Ответь информативно и точно, используя только предоставленную информацию.
Если в материалах нет точного ответа, скажи об этом.<end_of_turn>
<start_of_turn>model
"""
    
    return prompt

# 6. Генерация ответа
def generate_answer(prompt):
    """Генерируем ответ с помощью модели"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Извлекаем только ответ модели
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    
    # Очищаем от специальных токенов
    response = response.replace("<end_of_turn>", "").replace("<eos>", "").strip()
    
    return response

# 7. Основная функция RAG
def rag_pipeline(question, index, documents):
    """Полный пайплайн RAG"""
    # Шаг 1: Поиск релевантных документов
    relevant_docs = search_similar_documents(question, index, documents, top_k=3)
    
    # Шаг 2: Создание промпта
    prompt = create_prompt(question, relevant_docs)
    
    # Шаг 3: Генерация ответа
    answer = generate_answer(prompt)
    
    return answer, relevant_docs

# 8. Запуск системы
def main():
    # Инициализация
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("\n" + "="*60)
    print("RAG СИСТЕМА НА ОСНОВЕ РЕАЛЬНЫХ НОВОСТЕЙ")
    print("="*60)
    print(f"Загружено {len(documents)} новостных документов")
    print("Система готова! Введите ваш вопрос (или 'выход' для завершения):")
    
    # Сначала запускаем тестовые вопросы
    run_test_questions()
    
    # Интерактивный цикл
    while True:
        question = input("\n🎯 Ваш вопрос: ").strip()
        
        if question.lower() in ['выход', 'exit', 'quit']:
            print("До свидания!")
            break
            
        if not question:
            continue
            
        try:
            print("🔍 Ищем релевантные новости...")
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"\n🤖 Ответ: {answer}")
            print(f"\n📊 Использовано источников: {len(context_docs)}")
            print("📰 Релевантные новости:")
            for i, doc in enumerate(context_docs, 1):
                print(f"   {i}. {doc['text'][:120]}... (релевантность: {doc['score']:.3f})")
            
        except Exception as e:
            print(f"❌ Ошибка: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import json
from datasets import load_dataset

# 1. Загрузка моделей
print("Загружаем модели...")

# Модель для эмбеддингов (русский язык)
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Модель для генерации
model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Загрузка реального датасета новостей
def create_knowledge_base():
    """Загружаем реальный датасет новостей на русском языке"""
    print("Загружаем датасет")
    
    try:
        # Загружаем датасет
        dataset = load_dataset("zloelias/lenta-ru", split="train[:1000]")  # Берем первые 1000 новостей
        
        documents = []
        for item in dataset:
            # Извлекаем текст новости
            text = item.get('text', '')
            title = item.get('title', '')
            
            # Создаем документ из заголовка и начала текста
            if title and text:
                document = f"{title}. {text[:300]}..."  # Ограничиваем длину
                documents.append(document)
            elif text:
                documents.append(text[:400] + "...")
            elif title:
                documents.append(title)
        
        print(f"Загружено {len(documents)} новостей из датасета")
        return documents
        
    except Exception as e:
        print(f"Ошибка загрузки датасета: {e}")
        # Резервный вариант - возвращаем несколько примеров
        return [
            "Россия и Китай усиливают экономическое сотрудничество в условиях санкций.",
            "Центробанк России сохранил ключевую ставку на прежнем уровне.",
            "В Москве открылась новая ветка метро, соединяющая центр с спальными районами.",
            "Ученые разработали новую вакцину от гриппа с повышенной эффективностью.",
            "Цены на нефть выросли на мировых рынках из-за сокращения добычи.",
        ]

# Примеры вопросов для тестирования (адаптированные под новостную тематику)
test_questions = [
    "Какие последние новости о экономике России?",
    "Что происходит с курсом рубля?",
    "Какие новые проекты в метро Москвы?",
    "Какие научные разработки последнего времени?",
    "Как изменились цены на нефть?",
    "Какие международные отношения у России?",
    "Что нового в банковской системе?",
    "Какие события в культурной жизни?",
    "Какие спортивные новости?",
    "Что происходит в сфере технологий?",
    "Какие изменения в образовании?",
    "Какие медицинские новости?",
    "Какие политические события?",
    "Что нового в строительстве?",
    "Какие экологические проблемы обсуждаются?"
]

def run_test_questions():
    """Функция для тестирования системы на подготовленных вопросах"""
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("=" * 60)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ С РЕАЛЬНЫМИ НОВОСТЯМИ")
    print("=" * 60)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Вопрос: {question}")
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"🤖 Ответ: {answer}")
            print(f"📊 Использовано документов: {len(context_docs)}")
            print("📰 Источники:")
            for j, doc in enumerate(context_docs, 1):
                print(f"   {j}. {doc['text'][:100]}... (сходство: {doc['score']:.3f})")
            print("-" * 80)
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            print("-" * 80)

# 3. Создание векторной базы данных
def create_vector_db(documents):
    """Создаем векторную базу данных"""
    print("Создаем векторную базу из новостей...")
    
    # Получаем эмбеддинги для документов
    embeddings = embedding_model.encode(documents)
    
    # Создаем FAISS индекс
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (косинусное сходство)
    
    # Нормализуем векторы для косинусного сходства
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    print(f"Векторная база создана: {len(documents)} документов")
    return index, documents

# 4. Поиск релевантных документов
def search_similar_documents(query, index, documents, top_k=3):
    """Ищем наиболее релевантные документы для запроса"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Ищем похожие документы
    scores, indices = index.search(query_embedding, top_k)
    
    # Возвращаем найденные документы и их скоринги
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):
            results.append({
                'text': documents[idx],
                'score': scores[0][i]
            })
    
    return results

# 5. Создание промпта с контекстом
def create_prompt(question, context_docs):
    """Создаем промпт для модели с контекстом"""
    context = "\n".join([f"- {doc['text']}" for doc in context_docs])
    
    prompt = f"""<bos><start_of_turn>user
На основе следующих новостных материалов ответь на вопрос:

{context}

Вопрос: {question}

Ответь информативно и точно, используя только предоставленную информацию.
Если в материалах нет точного ответа, скажи об этом.<end_of_turn>
<start_of_turn>model
"""
    
    return prompt

# 6. Генерация ответа
def generate_answer(prompt):
    """Генерируем ответ с помощью модели"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Извлекаем только ответ модели
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    
    # Очищаем от специальных токенов
    response = response.replace("<end_of_turn>", "").replace("<eos>", "").strip()
    
    return response

# 7. Основная функция RAG
def rag_pipeline(question, index, documents):
    """Полный пайплайн RAG"""
    # Шаг 1: Поиск релевантных документов
    relevant_docs = search_similar_documents(question, index, documents, top_k=3)
    
    # Шаг 2: Создание промпта
    prompt = create_prompt(question, relevant_docs)
    
    # Шаг 3: Генерация ответа
    answer = generate_answer(prompt)
    
    return answer, relevant_docs

# 8. Запуск системы
def main():
    # Инициализация
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("\n" + "="*60)
    print("RAG СИСТЕМА НА ОСНОВЕ РЕАЛЬНЫХ НОВОСТЕЙ")
    print("="*60)
    print(f"Загружено {len(documents)} новостных документов")
    print("Система готова! Введите ваш вопрос (или 'выход' для завершения):")
    
    # Сначала запускаем тестовые вопросы
    run_test_questions()
    
    # Интерактивный цикл
    while True:
        question = input("\n🎯 Ваш вопрос: ").strip()
        
        if question.lower() in ['выход', 'exit', 'quit']:
            print("До свидания!")
            break
            
        if not question:
            continue
            
        try:
            print("🔍 Ищем релевантные новости...")
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"\n🤖 Ответ: {answer}")
            print(f"\n📊 Использовано источников: {len(context_docs)}")
            print("📰 Релевантные новости:")
            for i, doc in enumerate(context_docs, 1):
                print(f"   {i}. {doc['text'][:120]}... (релевантность: {doc['score']:.3f})")
            
        except Exception as e:
            print(f"❌ Ошибка: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import json
from datasets import load_dataset
import pickle
import os


print("Загружаем модели...")

# Модель для эмбеддингов (русский язык)
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Модель для генерации
model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Загрузка реального датасета новостей
def create_knowledge_base():
    """Загружаем реальный датасет новостей на русском языке"""
    print("Загружаем датасет...")
    
    try:
        # Загружаем датасет
        dataset = load_dataset("zloelias/lenta-ru", split="train[:1000]")  # Берем первые 1000 новостей
        
        documents = []
        for item in dataset:
            # Извлекаем текст новости
            text = item.get('text', '')
            title = item.get('title', '')
            
            # Создаем документ из заголовка и начала текста
            if title and text:
                document = f"{title}. {text[:300]}..."  # Ограничиваем длину
                documents.append(document)
            elif text:
                documents.append(text[:400] + "...")
            elif title:
                documents.append(title)
        
        print(f"Загружено {len(documents)} новостей из датасета")
        return documents
        
    except Exception as e:
        print(f"Ошибка загрузки датасета: {e}")
        # Резервный вариант - возвращаем несколько примеров
        return [
            "Россия и Китай усиливают экономическое сотрудничество в условиях санкций.",
            "Центробанк России сохранил ключевую ставку на прежнем уровне.",
            "В Москве открылась новая ветка метро, соединяющая центр с спальными районами.",
            "Ученые разработали новую вакцину от гриппа с повышенной эффективностью.",
            "Цены на нефть выросли на мировых рынках из-за сокращения добычи.",
        ]

# 3. Загрузка вопросов из pickle файла
def load_questions_from_pickle(filename="input.pickle"):
    """Загружаем вопросы из pickle файла"""
    try:
        if not os.path.exists(filename):
            print(f"Файл {filename} не найден. Использую тестовые вопросы.")
            return get_default_questions()
        
        with open(filename, 'rb') as f:
            questions_data = pickle.load(f)
        
        # Обрабатываем разные форматы pickle файла
        if isinstance(questions_data, list):
            # Если это простой список вопросов
            questions = questions_data
        elif isinstance(questions_data, dict):
            # Если это словарь, пытаемся извлечь вопросы
            if 'questions' in questions_data:
                questions = questions_data['questions']
            elif 'queries' in questions_data:
                questions = questions_data['queries']
            else:
                # Берем все строковые значения
                questions = [v for v in questions_data.values() if isinstance(v, str)]
        else:
            print(f"Неизвестный формат файла {filename}. Использую тестовые вопросы.")
            return get_default_questions()
        
        print(f"Загружено {len(questions)} вопросов из {filename}")
        return questions
        
    except Exception as e:
        print(f"Ошибка загрузки вопросов из {filename}: {e}")
        return get_default_questions()

def get_default_questions():
    """Возвращает вопросы по умолчанию если pickle файл недоступен"""
    return [
        "Какие последние новости о экономике России?",
        "Что происходит с курсом рубля?",
        "Какие новые проекты в метро Москвы?",
        "Какие научные разработки последнего времени?",
        "Как изменились цены на нефть?",
        "Какие международные отношения у России?",
        "Что нового в банковской системе?",
        "Какие события в культурной жизни?",
        "Какие спортивные новости?",
        "Что происходит в сфере технологий?"
    ]

def run_test_questions():
    """Функция для тестирования системы на вопросах из pickle"""
    # Загружаем вопросы из pickle
    test_questions = load_questions_from_pickle("input.pickle")
    
    # Загружаем базу знаний
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("=" * 60)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ С РЕАЛЬНЫМИ НОВОСТЯМИ")
    print("=" * 60)
    print(f"Будет обработано {len(test_questions)} вопросов")
    
    # Сохраняем результаты
    results = []
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Вопрос: {question}")
        try:
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"🤖 Ответ: {answer}")
            print(f"📊 Использовано документов: {len(context_docs)}")
            print("📰 Источники:")
            for j, doc in enumerate(context_docs, 1):
                print(f"   {j}. {doc['text'][:100]}... (сходство: {doc['score']:.3f})")
            
            # Сохраняем результат
            results.append({
                'question': question,
                'answer': answer,
                'sources': [doc['text'] for doc in context_docs],
                'scores': [doc['score'] for doc in context_docs]
            })
            
            print("-" * 80)
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            results.append({
                'question': question,
                'answer': f"Ошибка: {e}",
                'sources': [],
                'scores': []
            })
            print("-" * 80)
    
    # Сохраняем результаты в файл
    save_results(results)
    return results

def save_results(results, filename="rag_results.pickle"):
    """Сохраняем результаты работы RAG системы"""
    try:
        with open(filename, 'wb') as f:
            pickle.dump(results, f)
        print(f"\n💾 Результаты сохранены в {filename}")
        
        # Также сохраняем в читаемом формате
        with open("rag_results.txt", "w", encoding="utf-8") as f:
            f.write("РЕЗУЛЬТАТЫ RAG СИСТЕМЫ\n")
            f.write("=" * 50 + "\n")
            for i, result in enumerate(results, 1):
                f.write(f"\n{i}. ВОПРОС: {result['question']}\n")
                f.write(f"   ОТВЕТ: {result['answer']}\n")
                f.write(f"   ИСТОЧНИКИ: {len(result['sources'])}\n")
                for j, source in enumerate(result['sources'], 1):
                    f.write(f"      {j}. {source[:100]}...\n")
                f.write("-" * 80 + "\n")
        
        print(f"📄 Текстовый отчет сохранен в rag_results.txt")
        
    except Exception as e:
        print(f"Ошибка сохранения результатов: {e}")

# 4. Создание векторной базы данных
def create_vector_db(documents):
    """Создаем векторную базу данных"""
    print("Создаем векторную базу из новостей...")
    
    # Получаем эмбеддинги для документов
    embeddings = embedding_model.encode(documents)
    
    # Создаем FAISS индекс
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner Product (косинусное сходство)
    
    # Нормализуем векторы для косинусного сходства
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    print(f"Векторная база создана: {len(documents)} документов")
    return index, documents

# 5. Поиск релевантных документов
def search_similar_documents(query, index, documents, top_k=3):
    """Ищем наиболее релевантные документы для запроса"""
    # Получаем эмбеддинг запроса
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Ищем похожие документы
    scores, indices = index.search(query_embedding, top_k)
    
    # Возвращаем найденные документы и их скоринги
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):
            results.append({
                'text': documents[idx],
                'score': scores[0][i]
            })
    
    return results

# 6. Создание промпта с контекстом
def create_prompt(question, context_docs):
    """Создаем промпт для модели с контекстом"""
    context = "\n".join([f"- {doc['text']}" for doc in context_docs])
    
    # Адаптируем промпт под формат Qwen
    prompt = f"""<|im_start|>system
Ты - помощник, который отвечает на вопросы на основе предоставленных новостных материалов.
Отвечай точно и информативно, используя только предоставленную информацию.
Если в материалах нет точного ответа, скажи об этом.<|im_end|>
<|im_start|>user
На основе следующих новостных материалов ответь на вопрос:

{context}

Вопрос: {question}<|im_end|>
<|im_start|>assistant
"""
    
    return prompt

# 7. Генерация ответа
def generate_answer(prompt):
    """Генерируем ответ с помощью модели"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Извлекаем только ответ ассистента
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1].strip()
        if "<|im_end|>" in response:
            response = response.split("<|im_end|>")[0].strip()
    
    return response

# 8. Основная функция RAG
def rag_pipeline(question, index, documents):
    """Полный пайплайн RAG"""
    # Шаг 1: Поиск релевантных документов
    relevant_docs = search_similar_documents(question, index, documents, top_k=3)
    
    # Шаг 2: Создание промпта
    prompt = create_prompt(question, relevant_docs)
    
    # Шаг 3: Генерация ответа
    answer = generate_answer(prompt)
    
    return answer, relevant_docs

# 9. Запуск системы
def main():
    # Инициализация
    documents = create_knowledge_base()
    index, documents = create_vector_db(documents)
    
    print("\n" + "="*60)
    print("RAG СИСТЕМА НА ОСНОВЕ РЕАЛЬНЫХ НОВОСТЕЙ")
    print("="*60)
    print(f"Загружено {len(documents)} новостных документов")
    
    # Запускаем тестирование на вопросах из pickle
    results = run_test_questions()
    
    # Интерактивный цикл
    print("\n" + "="*60)
    print("ИНТЕРАКТИВНЫЙ РЕЖИМ")
    print("="*60)
    print("Введите ваш вопрос (или 'выход' для завершения):")
    
    while True:
        question = input("\n🎯 Ваш вопрос: ").strip()
        
        if question.lower() in ['выход', 'exit', 'quit']:
            print("До свидания!")
            break
            
        if not question:
            continue
            
        try:
            print("🔍 Ищем релевантные новости...")
            answer, context_docs = rag_pipeline(question, index, documents)
            print(f"\n🤖 Ответ: {answer}")
            print(f"\n📊 Использовано источников: {len(context_docs)}")
            print("📰 Релевантные новости:")
            for i, doc in enumerate(context_docs, 1):
                print(f"   {i}. {doc['text'][:120]}... (релевантность: {doc['score']:.3f})")
            
        except Exception as e:
            print(f"❌ Ошибка: {e}")

if __name__ == "__main__":
    main()